In [1]:
{-# LANGUAGE TypeOperators, FlexibleContexts, TypeFamilies #-}
import Prelude ()
import Data.Manifold.TreeCover
import Data.Random
import Data.Random.Manifold
import Data.Manifold
import Data.Manifold.Shade
import Data.Manifold.Web
import Data.Manifold.DifferentialEquation
import Math.LinearMap.Category
import Data.Function.Affine
import Data.VectorSpace
import Linear(V2(..), _x, _y)
import Data.Semigroup
import qualified Data.Foldable as Hask
import qualified Control.Monad as Hask
import Control.Lens

import Control.Category.Constrained.Prelude
import Control.Arrow.Constrained
:opt no-lint  -- lint gives bogus warnings with constrained-categories

From [diagrams](http://projects.haskell.org/diagrams):

In [2]:
import Diagrams.Prelude (p2, circle, (&), (^&), moveTo, opacity, fromVertices)

From [dynamic-plot](http://hackage.haskell.org/package/dynamic-plot):

In [3]:
import Graphics.Dynamic.Plot.R2

type T = ℝ
type X = ℝ

viewRange = plot [forceXRange (-2,4), forceYRange (-1,3)]

wPlot' :: PointsWeb T (Shade' X) -> DynamicPlottable
wPlot' wb = plot
            [ shapePlot $ fromVertices [p2 (t₁,x₁), p2 (t₂,x₂)]
            | ((t₁,Shade' x₁ _),(t₂,Shade' x₂ _)) <- edg ]
 where edg = webEdges wb

In [4]:
μ :: LocalLinear T X +> X
μ = arr.LinearFunction $ \d -> d$1

deq :: ODE T X
deq = constLinearODE μ

In [5]:
tf :: Needle T -> PointsWeb T (Shade' X)
tf δt₀ = fromWebNodes euclideanMetric $ 
        (0, 1|±|[0.01])
        : [(t, 1|±|[3]) | t<-[δt₀, 2*δt₀ .. 1.2] ]

forM_ [ Hask.toList $ iterateFilterDEqn_static (inconsistencyAware intersectShade's) id deq (tf 0.05)
      , Hask.toList $ iterateFilterDEqn_static_selective
                          (inconsistencyAware intersectShade's) id (euclideanVolGoal 0.001) deq (tf 0.05)
      , iterateFilterDEqn_adaptive euclideanMetric AbortOnInconsistency
                   deq (euclideanVolGoal 0.001) (tf 0.2)]
  $ \tfs -> do
    plotWindow
       [ plot ((1^&exp 1) :± [0.1^&0, 0^&0.1] :: Shade ℝ²) -- Euler's number as reference for x=1
       , plotLatest [plot st & legendName (show i) | (i,st) <- zip [0..] tfs]
         --  & plotDelay 2
       , plot ((0.4^&exp 0.4) :± [0.1^&0, 0^&0.1] :: Shade ℝ²) -- Euler's number as reference for x=1
       ]

Static resolution:
![filtering the exponential function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential.gif)
Adaptive resolution:
![filtering the exponential function as a solution to an ODE (adaptive resolution)](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-exponential-adaptive.gif)

In [6]:
import qualified Control.Comonad.Cofree as Cofree
import Data.Foldable(Foldable)
import Data.List.NonEmpty(NonEmpty(..))
import Control.Monad.Trans.Except

tf_bad :: Needle T -> PointsWeb T (Shade' X)
tf_bad δt₀ = fromWebNodes euclideanMetric $ 
        (0, 0.5|±|[0.01]) : [(t, (1-t)|±|[1]) | t<-[δt₀, 2*δt₀ .. 1.2] ]

tfs_inconsistent = iterateFilterDEqn_static (indicateInconsistencies intersectShade's)
                     id deq (tf_bad 0.05)
plotWindow [ plotLatest [plot st & legendName (show i) | (i,st) <- zip [0..] $ Hask.toList tfs_inconsistent] ]

findErr :: (Hask.Monad m, Hask.Foldable m)
          => Cofree.Cofree m a -> m ()
findErr (a Cofree.:< q) = case Hask.toList q of
        [] -> const () <$> q
        l  -> foldr1 (>>) $ findErr<$>l

case runExcept $ findErr tfs_inconsistent of Left e -> print e

GraphWindowSpecR2{lBound=-0.20000000000000012, rBound=1.4000000000000008, bBound=-0.3916666666666674, tBound=1.1416666666666668, xResolution=640, yResolution=480}

PropagationInconsistency {_inconsistentPropagatedData = [(0.8500000000000003,0.98|±|[0.05])], _inconsistentAPrioriData = 1.071|±|[7e-3]}

In [7]:
type Y = ℝ
type XY = ℝ²

μ₂ :: LocalLinear T XY +> XY
μ₂ = arr.LinearFunction $ ($1) >>> \(V2 dx dy) -> V2 dy (-dx)

deq₂ :: ODE T XY
deq₂ = constLinearODE μ₂

In [8]:
tf₂ :: Needle T -> PointsWeb T (Shade' XY)
tf₂ δt₀ = fromWebNodes euclideanMetric
        [(t, (1^&0)|±|[0^&δ, δ^&0]) | t<-(^2)<$>[0, sqrt δt₀ .. 2], let δ = 0.0001 + 4*t ]

forM_ [ Hask.toList $ iterateFilterDEqn_static (inconsistencyAware intersectShade's) id deq₂ (tf₂ 0.004)
      , Hask.toList $ iterateFilterDEqn_static_selective (inconsistencyAware intersectShade's) id
                         (euclideanVolGoal 0.001) deq₂ (tf₂ 0.004)
      , iterateFilterDEqn_adaptive euclideanMetric AbortOnInconsistency
              deq₂ (euclideanVolGoal 0.001) (tf₂ 0.01) ]
   $ \tfs₂ -> do
 plotWindow [ continFnPlot sin, continFnPlot cos
  , plotLatest [ plot [fmap fst tffacts, fmap snd tffacts] & legendName (show i)
               | (i,tf') <- zip [0..] tfs₂
               , let tffacts = fmap factoriseShade (coerceShade <$> tf' :: PointsWeb T (Shade' (X,Y))) ] ]

![filtering the sin and cosine function as a solution to an ODE](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-trigonometric.png)
![filtering the sin and cosine function as a solution to an ODE (adaptive resolution)](https://raw.githubusercontent.com/leftaroundabout/manifolds/master/manifolds/images/examples/ODE-solution-filter/simple-trigonometric-adaptive.png)

In [9]:
--case runExcept $ findErr (iterateFilterDEqn_static (indicateInconsistencies intersectShade's) id deq₂ (tf₂ 0.004)) of
  -- Left e -> print e

--[(x, cos x) | x <- (^2)<$>[0, sqrt 0.004 .. sqrt 0.1]]

In [10]:
import Math.LinearMap.Category.Derivatives

deq₃Init :: T -> Shade' XY
deq₃Init t = (f t ^& 2)|±|[1e-6^&0, 0^&δ]
 where δ | t > -4      = 0.9
         | otherwise  = 1e-6
       f t = sin (t + 3*sin (t/2))

-- Simple “differentials equalisation” system: make it so the functions x(t) and y(t) move in parallel.
deq₃ :: DifferentialEqn T XY
deq₃ (Shade (t,_) _) = LocalDifferentialEqn {
   _rescanDifferentialEqn = \d⁰ d¹ d²
      -> let x' = projectShade (lensEmbedding (1*∂_x/∂id)) d¹ :: Shade' ℝ
             y' = projectShade (lensEmbedding (1*∂_y/∂id)) d¹ :: Shade' ℝ
         in ( (if t>0 then intersectShade's . (:|[d⁰]) else return)
                $ deq₃Init t
            , return $
                  embedShade (lensEmbedding (1*∂_y/∂id)) x'
              {-:|[ embedShade (lensEmbedding (1*∂_x/∂id)) y' ]-})
   }

In [11]:
import Control.Applicative ((<|>))

tf₃ :: Needle T -> PointsWeb T (Shade' XY)
tf₃ δt₀ = fromWebNodes euclideanMetric
        [ (t, deq₃Init t)
        | t <- subtract (δt₀/2) . (^2)<$>[0, sqrt δt₀ .. 3] ]

tfs₃ = iterateFilterDEqn_static (indicateInconsistencies $ \l -> intersectShade's l)
                     id deq₃ (tf₃ 0.01)
tfs₃_s = iterateFilterDEqn_static_selective (indicateInconsistencies $ \l -> intersectShade's l)
                     id (euclideanVolGoal 0.001) deq₃ (tf₃ 0.01)

forM_ [ Hask.toList tfs₃, Hask.toList tfs₃_s ]
   $ \tfs -> do
 plotWindow [
    plotLatest [ plot [fmap fst tffacts, fmap snd tffacts] & legendName (show i)
               | (i,tf') <- zip [0..] tfs
               , let tffacts = fmap factoriseShade (coerceShade <$> tf' :: PointsWeb T (Shade' (X,Y))) ] ]

In [12]:
case runExcept $ findErr tfs₃_s of Left e -> print e

PropagationInconsistency {_inconsistentPropagatedData = [(1.6850000000000007,V2 -0.2627938 1.89|±|[V2 5e-7 0.00,V2 0 0.12]),(1.2050000000000003,V2 -0.2614914 1.83|±|[V2 5e-7 -0.00,V2 0 0.10])], _inconsistentAPrioriData = V2 -0.2627938 1.86|±|[V2 5e-7 -0.00,V2 0 0.11]}